In [10]:
from vod.frame import FrameDataLoader
from vod.frame import FrameTransformMatrix
from vod.configuration import KittiLocations

FrameTransformMatrix(FrameDataLoader(kitti_locations = KittiLocations(root_dir='/mnt/data/fangqiang/view_of_delft/'), 
                                    frame_number='00000')).t_lidar_camera

array([[-0.00798023,  0.11849694,  0.99292227,  0.964356  ],
       [-0.99985405, -0.0159445 , -0.00613307,  0.13801578],
       [ 0.0151049 , -0.99282638,  0.11860694, -0.35144845],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [6]:
FrameTransformMatrix(FrameDataLoader(kitti_locations = KittiLocations(root_dir='/mnt/data/fangqiang/view_of_delft/'), 
                                    frame_number='00000')).camera_projection_matrix

array([[1.4954686e+03, 0.0000000e+00, 9.6127246e+02, 0.0000000e+00],
       [0.0000000e+00, 1.4954686e+03, 6.2489594e+02, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00]],
      dtype=float32)

In [7]:

FrameTransformMatrix(FrameDataLoader(kitti_locations = KittiLocations(root_dir='/mnt/data/fangqiang/view_of_delft/'), 
                                    frame_number='00000')).t_camera_lidar

array([[-0.0079802 , -0.99985409,  0.0151049 ,  0.15099999],
       [ 0.118497  , -0.0159445 , -0.9928264 , -0.461     ],
       [ 0.99292243, -0.0061331 ,  0.1186069 , -0.91500002],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [4]:

FrameTransformMatrix(FrameDataLoader(kitti_locations = KittiLocations(root_dir='/mnt/data/fangqiang/view_of_delft/'), 
                                    frame_number='00000')).t_lidar_camera

array([[-0.00798023,  0.11849694,  0.99292227,  0.964356  ],
       [-0.99985405, -0.0159445 , -0.00613307,  0.13801578],
       [ 0.0151049 , -0.99282638,  0.11860694, -0.35144845],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [18]:

math.atan2(0, 1)

0.0

In [31]:
import numpy as np
import math
# Transformation matrices provided
t_lidar_camera = FrameTransformMatrix(FrameDataLoader(kitti_locations = KittiLocations(root_dir='/mnt/data/fangqiang/view_of_delft/'), 
                                    frame_number='00000')).t_camera_lidar

t_lidar_radar = FrameTransformMatrix(FrameDataLoader(kitti_locations = KittiLocations(root_dir='/mnt/data/fangqiang/view_of_delft/'), 
                                    frame_number='00000')).t_radar_lidar
def cartesian_to_spherical_radar(x, y, z):
    azimuth = math.degrees(math.atan2(z, x))  # Azimuth in degrees
    elevation = math.degrees(math.atan2(y, x))  # Elevation in degrees
    return azimuth, elevation
def cartesian_to_spherical_cam(x, y, z):
    azimuth = math.degrees(math.atan2(y, z))  # Azimuth in degrees
    elevation = math.degrees(math.atan2(x, z))  # Elevation in degrees
    return azimuth, elevation

# Function to transform points using a transformation matrix
def transform_points(points, transformation_matrix):
    # Adding a column of ones for homogeneous coordinates
    homogeneous_points = np.hstack((points, np.ones((points.shape[0], 1))))
    transformed_points = np.dot(homogeneous_points, transformation_matrix.T)
    return transformed_points[:, :3]  # Exclude the homogeneous coordinate

point_cloud = np.array([[100,0,0]])


# Function to filter points based on FoV
def filter_points_fov_radar(points, horizontal_fov, vertical_fov):
    filtered_points = []
    for point in points:
        tf_point = transform_points(point, t_lidar_radar)
        az, el = cartesian_to_spherical_radar(point[0], point[1], point[2])
        if -horizontal_fov <= az <= horizontal_fov and -vertical_fov <= el <= vertical_fov:
            filtered_points.append(point)
    return np.array(filtered_points)

def filter_points_fov_camera(points, horizontal_fov, vertical_fov):
    filtered_points = []
    for point in points:
        tf_point = transform_points(point, t_lidar_camera)
        az, el = cartesian_to_spherical_cam(tf_point[0], tf_point[1], tf_point[2])
        print(az,el)
        if -horizontal_fov <= az <= horizontal_fov and -vertical_fov <= el <= vertical_fov:
            filtered_points.append(point)
    return np.array(filtered_points)


# Filter points for camera and radar
filtered_points_camera = filter_points_fov_camera(points_camera_frame, 32, 22)
filtered_points_radar = filter_points_fov_radar(points_radar_frame, 60, 15)

# Combine the results to find points within both FoVs
# Points need to be within both the camera and radar FoVs
combined_filtered_points = np.array([point for point in filtered_points_camera if point in filtered_points_radar])

len(combined_filtered_points), len(filtered_points_camera), len(filtered_points_radar)  # Number of points in each category


6.60348539150808 -0.3768247904909224


(0, 1, 1)

In [33]:
filtered_points_radar

array([[97.48993562, -0.64651248,  0.26613417]])

In [32]:
filtered_points_camera

array([[-0.64702005, 11.38869992, 98.37724251]])